In [14]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_bayesian_regret_fixed_cost_runs(acq, dim, kernel, lengthscale):
    runs = api.runs(path="ziv-scully-group/PandoraBayesOPT", filters={
        "sweep": "0329pw34",
        "config.amplitude": 1,
        "config.dim": dim,
        "config.kernel": kernel,
        "config.lengthscale": lengthscale,  
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        summary_metric_keys = ["global optimum value"]
        summary_history = run.scan_history(keys = summary_metric_keys, page_size=1_000_000_000)
        metrics.update({k: [d[k] for d in summary_history] for k in summary_metric_keys})
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [15]:
acquisition_functions = { 
    'KnowledgeGradient':'KnowledgeGradient',
    'UpperConfidenceBound': 'UpperConfidenceBound',
    'MultiStepLookaheadEI':'MultiStepLookaheadEI',
    }
dimensions = [16]
kernels = ["Matern52"]
lengthscales = [0.1]

In [16]:
grouped_runs = {(a,d,k,l): load_bayesian_regret_fixed_cost_runs(a,d,k,l) for a in acquisition_functions.keys() for d in (dimensions) for k in (kernels) for l in (lengthscales)}

0it [00:00, ?it/s]


In [13]:
for a in acquisition_functions.keys():
    for d in dimensions:
        for k in kernels:
            for l in lengthscales:
                config_and_metrics_per_seed = grouped_runs[a,d,k,l]
        
                cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T
                best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T
                # Handling potential empty arrays
                if cumulative_cost_per_seed.size == 0 or best_observed_per_seed.size == 0:
                    continue  # Skip this iteration if there's no data
        
                print(a, d, k, l, best_observed_per_seed.shape)
        
                best_25 = np.quantile(best_observed_per_seed, 0.25, axis=1)
                best_50 = np.quantile(best_observed_per_seed, 0.5, axis=1)
                best_75 = np.quantile(best_observed_per_seed, 0.75, axis=1)
        
                output = np.stack((cumulative_cost_per_seed.mean(axis=1), best_25, best_50, best_75),axis=-1)
        
                np.savetxt(f"results/fixed_amplitude_best_observed/FixedAmplitude_FixedCost_d{d}_{k}_ls{l}_{acquisition_functions[a]}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

UpperConfidenceBound 16 Matern52 0.1 (401, 16)


In [4]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_bayesian_regret_fixed_cost_runs(acq, dim, kernel, lengthscale):
    runs = api.runs(path="ziv-scully-group/PandoraBayesOPT", filters={
        "sweep": "h6skp9f3",
        "config.amplitude": 1,
        "config.dim": dim,
        "config.kernel": kernel,
        "config.lengthscale": lengthscale,  
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        summary_metric_keys = ["global optimum value"]
        summary_history = run.scan_history(keys = summary_metric_keys, page_size=1_000_000_000)
        metrics.update({k: [d[k] for d in summary_history] for k in summary_metric_keys})
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [5]:
acquisition_functions = { 
    'MaxValueEntropy':'MaxValueEntropy'
    }
dimensions = [16]
kernels = ["Matern52"]
lengthscales = [0.1]

In [6]:
grouped_runs = {(a,d,k,l): load_bayesian_regret_fixed_cost_runs(a,d,k,l) for a in acquisition_functions.keys() for d in (dimensions) for k in (kernels) for l in (lengthscales)}

100%|██████████| 16/16 [00:18<00:00,  1.16s/it]


In [7]:
for a in acquisition_functions.keys():
    for d in dimensions:
        for k in kernels:
            for l in lengthscales:
                config_and_metrics_per_seed = grouped_runs[a,d,k,l]
        
                cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T
                best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0]).T
                # Handling potential empty arrays
                if cumulative_cost_per_seed.size == 0 or best_observed_per_seed.size == 0:
                    continue  # Skip this iteration if there's no data
        
                print(a, d, k, l, best_observed_per_seed.shape)
        
                best_25 = np.quantile(best_observed_per_seed, 0.25, axis=1)
                best_50 = np.quantile(best_observed_per_seed, 0.5, axis=1)
                best_75 = np.quantile(best_observed_per_seed, 0.75, axis=1)
        
                output = np.stack((cumulative_cost_per_seed.mean(axis=1), best_25, best_50, best_75),axis=-1)
        
                np.savetxt(f"results/fixed_amplitude_best_observed/FixedAmplitude_FixedCost_d{d}_{k}_ls{l}_{acquisition_functions[a]}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

MaxValueEntropy 16 Matern52 0.1 (401, 16)
